# Merging data in pandas

You can use `pandas` to join relational data -- datasets that are intended to go together. You can also use `pandas` for enterprise joins -- experimentally merging datasets that aren't _necessarily_ intended to go together to help you answer reporting questions. ("How many of our elected officials are on a list of delinquent taxpayers?" or whatever.)

The method we'll be using here is [`merge()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html).

Let's start with a simple example: Merging two files of publicly available voter information from Liberty County, Florida. The files live here:

- `../data/voter-details.csv` -- A mildly anonymized subset of voter registration details (one row = one voter)
- `../data/LIB_H_20180509.txt` -- Voter history (one row = one vote)

We're gonna join on the `voterid` column.

First, we need to import pandas:

In [28]:
import pandas as pd

Next, let's read in the details file. We're going to pass the `read_csv()` method three arguments:
- `'../data/voter-details.csv'`: The path to the file
- `dtype={'voterid': str}`: The values in the `voterid` column should be interpreted as strings (this is the column we're going to join on later)
- `parse_dates=['birthdate', 'regdate']`: Parse these columns as dates

In [29]:
df_details = pd.read_csv('../data/voter-details.csv',
                         dtype={'voterid': str},
                         parse_dates=['birthdate', 'regdate'])

In [30]:
df_details.head()

,county,voterid,res_city,gender,race,birthdate,regdate,pty_aff,pct,voter_status,congress,house,senate,county_comm,school_board
0,LIB,119906189,BRISTOL,M,3,1992-06-14,2012-07-16,DEM,1,ACT,2,7,3,1,1
1,LIB,105265678,BRISTOL,F,5,1953-07-09,1999-08-16,DEM,8,ACT,2,7,3,5,5
2,LIB,105263429,HOSFORD,F,5,1941-09-12,1987-02-20,REP,6,ACT,2,7,3,5,5
3,LIB,100730611,BRISTOL,F,5,1981-09-24,2005-01-06,NPA,4,ACT,2,7,3,3,3
4,LIB,120366681,HOSFORD,M,5,1942-12-23,2012-10-08,NPA,5,ACT,2,7,3,4,4


Now let's read in the history file. This time, we're going to pass the `read_csv()` method _five_ arguments:
- `'../data/LIB_H_20180509.txt'`: The path to the file
- `sep='\t'`: The values in the file are separated by tabs
- `names=['county', 'voterid', 'election_date', 'election_type', 'ballot_type'],`: A list of column headers
- `dtype={'voterid': str}`: The values in the `voterid` column should be interpreted as strings (this is the column we're going to join on later)
- `parse_dates=['election_date]`: Parse the values in this column as dates

In [31]:
df_history = pd.read_csv('../data/LIB_H_20180509.txt',
                         sep='\t',
                         names=['county', 'voterid', 'election_date', 'election_type', 'ballot_type'],
                         dtype={'voterid': str},
                         parse_dates=['election_date'])

In [32]:
df_history.head()

,county,voterid,election_date,election_type,ballot_type
0,LIB,100672154,2008-08-26,PRI,Y
1,LIB,100672154,2012-11-06,GEN,E
2,LIB,102529531,2014-08-26,PRI,A
3,LIB,102531298,2016-03-15,PPP,Y
4,LIB,102537291,2016-03-15,PPP,Y


Now we're ready to merge. We're going to use the [`merge()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) method, passing it four arguments:
- `df_history`: The "left" table to join
- `df_details`: The "right" table to join
- `on='voterid'`: Join on the `voterid` column -- this works because the name of the column in both tables is "voterid." If the column names were _different_, you would specify the `left_on` and `right_on` arguments instead
- `how='left'`: We're doing a left join here

(If you need a refresher on joins, [Leigh Tami has drawn my current favorite visual explainer](https://twitter.com/leigh_tami18/status/1021471889309487105).)

In [33]:
merged = pd.merge(df_history,
                  df_details,
                  on='voterid',
                  how='left')

In [34]:
merged.head()

,county_x,voterid,election_date,election_type,ballot_type,county_y,res_city,gender,race,birthdate,regdate,pty_aff,pct,voter_status,congress,house,senate,county_comm,school_board
0,LIB,100672154,2008-08-26,PRI,Y,LIB,HOSFORD,F,5.0,1979-12-18,1998-01-23,DEM,5.0,ACT,2.0,7.0,3.0,4.0,4.0
1,LIB,100672154,2012-11-06,GEN,E,LIB,HOSFORD,F,5.0,1979-12-18,1998-01-23,DEM,5.0,ACT,2.0,7.0,3.0,4.0,4.0
2,LIB,102529531,2014-08-26,PRI,A,LIB,HOSFORD,M,5.0,1974-12-08,2000-05-22,DEM,6.0,ACT,2.0,7.0,3.0,5.0,5.0
3,LIB,102531298,2016-03-15,PPP,Y,LIB,HOSFORD,F,5.0,1974-08-22,2002-12-20,REP,6.0,ACT,2.0,7.0,3.0,5.0,5.0
4,LIB,102537291,2016-03-15,PPP,Y,LIB,HOSFORD,F,3.0,1976-11-02,1995-04-07,DEM,5.0,ACT,2.0,7.0,3.0,4.0,4.0


... and now you're ready to do some analysis. Let's break down Liberty County voters in the 2016 general election by race and party affiliation, for kicks:

In [35]:
# filter to get just votes in the 2016 general
votes_2016_gen = merged[merged['election_date'] == '2016-11-08']

# run a pivot table!
voters_2016_gen_grouped = pd.pivot_table(votes_2016_gen,
                                         index='race',
                                         values='voterid',
                                         aggfunc='count',
                                         columns='pty_aff').fillna(0).reset_index()

In the data, the voter's race/ethnicity is [coded as numbers](http://dos.myflorida.com/media/696057/voter-extract-file-layout.pdf). This is a great use for our good friend the dictionary -- the keys will be the numeric codes, the values will be the human-readable words.

Then we'll use the `apply()` method, with a _lambda expression_, to look up the race in the dictionary based on its numeric code.

👉 For more information on dictionaries, [check out this notebook](Python%20data%20types%20and%20basic%20syntax.ipynb#Dictionaries).

👉 For more information on the `apply()` method, [check out this notebook](Using%20the%20apply%20method%20in%20pandas.ipynb).

👉 For more information on lamda expressions, [check out this notebook](Functions.ipynb#Lambda-expressions).

In [36]:
# use a dictionary to map race codes to their values
# http://dos.myflorida.com/media/696057/voter-extract-file-layout.pdf
races = {
    1: 'American Indian or Alaskan Native',
    2: 'Asian Or Pacific Islander',
    3: 'Black, Not Hispanic',
    4: 'Hispanic',
    5: 'White, Not Hispanic',
    6: 'Other',
    7: 'Multi‐racial',
    9: 'Unknown'
}

# add a new column `race_verbose`
# using the `apply()` method and a lambda function to pull the value
# out of the `races` dictionary
voters_2016_gen_grouped['race_verbose'] = voters_2016_gen_grouped.race.apply(lambda x: races[x])

In [37]:
voters_2016_gen_grouped

pty_aff,race,DEM,LPF,NPA,REF,REP,race_verbose
0,1.0,20.0,0.0,3.0,0.0,10.0,American Indian or Alaskan Native
1,2.0,8.0,0.0,1.0,0.0,1.0,Asian Or Pacific Islander
2,3.0,259.0,0.0,12.0,0.0,3.0,"Black, Not Hispanic"
3,4.0,13.0,0.0,0.0,0.0,5.0,Hispanic
4,5.0,2141.0,1.0,150.0,1.0,574.0,"White, Not Hispanic"
5,6.0,3.0,0.0,2.0,0.0,0.0,Other
6,7.0,4.0,0.0,1.0,0.0,0.0,Multi‐racial
7,9.0,2.0,0.0,3.0,0.0,0.0,Unknown


### Enterprise joins

Next, let's take a look at a common data task in a newsroom: The enterprise join.

Our question: How many registered lobbyists in Florida are also admitted to practice law, according to the Florida Bar? Our goal is to come up with a solid estimate.

We're going to look at:
- A data file of Florida lobbyists, [pulled from the government website](https://www.floridalobbyist.gov/LobbyistInformation/DownloadData), and
- A data file of Florida attorneys, sourced from [the Florida Bar Association](https://www.floridabar.org/directories/find-mbr/)

... and see if we can find some matches.

First, let's load up the lobbyist data, which lives in a tab (`\t`)-delimited text file. We're going to specify that the 'ZIP' column is a string, parse values in the `Eff Date.1` column as dates and `skiprows=2` -- just what it sounds like (there's some cruft at the top of our file).

(Reminder: You can find a full list of arguments for the `read_csv()` method [in the pandas documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html).)

In [38]:
fl_lobbyists = pd.read_csv('../data/fl-llob.txt',
                           sep='\t',
                           skiprows=2,
                           dtype={'ZIP': str},
                           parse_dates=['Eff Date.1'])

In [39]:
fl_lobbyists.head()

,Last Name,First Name,Middle Name/Initial,Suffix,Address,Address.1,Address.2,City,State,ZIP,...,Address.6,City.2,State.2,ZIP.2,Country.1,Cntry Prefix,Phone.1,Ext.1,Eff Date.1,WD Date.1
0,Aaron,Lisa,NaN,NaN,7101 Sleepy Hollow Circle,NaN,NaN,Tallahassee,FL,32312,...,NaN,Tallahassee,FL,32312,NaN,NaN,(850) 321-7642,NaN,2018-01-03,NaN
1,Aaron,Lisa,NaN,NaN,7101 Sleepy Hollow Circle,NaN,NaN,Tallahassee,FL,32312,...,NaN,Tallahassee,FL,32312,NaN,NaN,(850) 321-7642,NaN,2018-01-03,NaN
2,Aaron,Lisa,NaN,NaN,7101 Sleepy Hollow Circle,NaN,NaN,Tallahassee,FL,32312,...,NaN,Tallahassee,FL,32312,NaN,NaN,(850) 321-7642,NaN,2018-01-03,NaN
3,Aaron,Lisa,NaN,NaN,7101 Sleepy Hollow Circle,NaN,NaN,Tallahassee,FL,32312,...,NaN,Tallahassee,FL,32312,NaN,NaN,(850) 321-7642,NaN,2018-01-03,NaN
4,Aaron,Lisa,NaN,NaN,7101 Sleepy Hollow Circle,NaN,NaN,Tallahassee,FL,32312,...,NaN,Tallahassee,FL,32312,NaN,NaN,(850) 321-7642,NaN,2018-01-03,NaN


Each record in the lobbying database is a lobbying relationship, so individual lobbyists show up multiple times. Let's select just the data in columns that identify the lobbyists and [drop duplicates](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.drop_duplicates.html).

In [40]:
columns_of_interest = ['Last Name', 'First Name', 'Middle Name/Initial', 'Suffix',
                       'Address', 'Address.1', 'Address.2', 'City',  'State', 'ZIP']

fl_lobbyists_unique = fl_lobbyists[columns_of_interest].drop_duplicates()

In [41]:
# how many did we drop?
print(len(fl_lobbyists))
print(len(fl_lobbyists_unique))

11296
1865


Now, let's read in the lawyer data, again specifying that the column with zipcodes should come in as a string.

In [42]:
fl_lawyers = pd.read_csv('../data/fl-attorneys-with-zip.csv',
                         dtype={'zip': str})

In [43]:
fl_lawyers.head()

,name,img,bar_no,url,status,contact,zipplus4,ZIP
0,Sarah Finney Kjellin,NaN,59126,https://www.floridabar.org/directories/find-mb...,Eligible to Practice Law in Florida,"Conroy Simberg Atrium Building, Suit 105 325 J...",32303-4113,32303
1,Katherine Marie Kjolhede,NaN,525375,https://www.floridabar.org/directories/find-mb...,Elected Status: Inactive,"Ford Credit 1 American Rd Dearborn, MI 48126-2701",48126-2701,48126
2,Marla Shomer Klaas,NaN,191670,https://www.floridabar.org/directories/find-mb...,Elected Status: Inactive,"Union Bank, N.A. Marla S Klaas APC 7061 Heron ...",92011-3975,92011
3,Roger W Klaffka,NaN,1004166,https://www.floridabar.org/directories/find-mb...,Eligible to Practice Law in Florida,"Mr. Roger W. Klaffka PO Box 26614 Tampa, FL 33...",33623-6614,33623
4,Kevin D Klagge,NaN,99502,https://www.floridabar.org/directories/find-mb...,Eligible to Practice Law in Florida,"Kalis, Kleiman & Wolfe 7320 Griffin Rd Ste 109...",33314-4105,33314


Great! Now we're ready to start exploring potential ways to merge this data. This is as much art as science: Too restrictive in your criteria and you'll miss matches; too broad and you'll have lots of false positives. You're trying to find a reasonable middle ground, and you'll likely couch your findings in phrases like "as many as," "more than," etc. to accurately reflect the precision of this kind of analysis.

For this case, I might start by matching on last name and the first letter of the first name to see where that gets me.

The lawyer data doesn't have the last name isolated, though -- it just has the full name in one field. So we'll have to do that manually. A quick and dirty way to extract the last name: Use [`split()`](https://docs.python.org/3/library/stdtypes.html#str.split) to break up the name into a list of pieces on spaces, then grab the last item in that list. We'll also uppercase the result for matching purposes later.

This approach will result in misses and false positives, however -- "Jr." and "III" and "van de" names. A more robust solution would be to use a library like [`probablepeople`](https://parserator.datamade.us/probablepeople) to parse the names into their constituent parts, but we'll leave that as an _exercise for the reader_, as the kids say.

In the `fl_lawyers` data frame, let's create a new column, `last`, that grabs "everything to the right of the final space" in the name field and makes it uppercase. We're going to use `apply()` with a _lambda expression_ to do this.

In [44]:
# inside apply(), a lambda expression that will
# split() on spaces, then take the last item [-1] of the resulting list
# and make it uppercase with upper()
fl_lawyers['last'] = fl_lawyers['name'].apply(lambda x: x.rsplit(' ')[-1].upper())

For both data frames, we'll create a new column, `first_l`, that just has the first letter of the first name. Again, we'll use `apply()` with a lambda expression. We're going to take advantage of the fact that [Python strings are iterable](Python%20data%20types%20and%20basic%20syntax.ipynb#for-loops), just like lists, which means they support _slicing_, and so our lambda expression is gonna be simple: Just take the first `[0]` character of the string. Just to be sure, we're also going to use `upper()` to make sure that the first letter is uppercase.

(Here's a basic example of string slicing:

```python
my_name = 'Cody'
print(my_name[0])
# => 'C'
```
)

In [45]:
fl_lawyers['first_l'] = fl_lawyers['name'].apply(lambda x: x[0].upper())
fl_lobbyists_unique['first_l'] = fl_lobbyists_unique['First Name'].apply(lambda x: x[0].upper())

One last thing to do: Create a new column in the lobbying data frame, `last`, that takes the person's last name and converts it to uppercase.

The reason we're doing all this upcasing is that equality tests on strings are case sensitive. For example:

In [46]:
'SMITH' == 'SMITH'

True

In [47]:
'SMITH' == 'Smith'

False

... so we're just making sure that we're doing apples-to-apples matching. For this one, we don't need to use `apply()` with a lambda expression -- we can just access the [`str`](https://pandas.pydata.org/pandas-docs/stable/text.html) attribute of the column and call the `upper()` method on it.

In [48]:
fl_lobbyists_unique['last'] = fl_lobbyists_unique['Last Name'].str.upper()

In [49]:
last_first_i = pd.merge(fl_lobbyists_unique,
                        fl_lawyers,
                        on=['last', 'first_l'],
                        how='left')

In [50]:
len(last_first_i)

7421

So: 7,421 records seems like a lot to go through manually. But maybe not! Maybe you have two dozen people to help and it's critical to your story that you miss as few people as possible. But probably we can do better. Let's try matching on those criteria, plus ZIP code.

In [51]:
last_first_i_zip = pd.merge(fl_lobbyists_unique,
                            fl_lawyers,
                            on=['last', 'first_l', 'ZIP'],
                            how='left')

In [52]:
len(last_first_i_zip)

1875

Better. Additional ideas for further refining the results:
- Using something like [`usaddress`](https://github.com/datamade/usaddress) to parse the addresses into their constituent parts and matching on, say, street
- As mentioned, using something like [`probablepeople`](https://parserator.datamade.us/probablepeople) to handle name parsing on the name field in the lawyer data frame
- Matching on first names, not just first initial
- Geocoding the addresses and matching on coordinates within a given tolerance

At any rate, once you get to a place where you feel comfortable enough to start hand-checking, then it's a matter of _notebook-assisted reporting_ to verify each potential match.

### Further reading

- [pandas documentation: joins](https://pandas.pydata.org/pandas-docs/stable/comparison_with_sql.html#join)
- [Merge and Join DataFrames with Pandas in Python](https://www.shanelynn.ie/merge-join-dataframes-python-pandas-index-1/)
- [Join and merge pandas dataframe](https://chrisalbon.com/python/data_wrangling/pandas_join_merge_dataframe/)